In [101]:
import torch
import torch.optim as optim
import numpy as np


n = 2
k = 2

quantities = [[10, 20], [60, 40]]  # Example initial quantities for each AMM
# 
# quantities = [[10, 20, 10], [60, 40, 100], [100, 150, 190], [5, 1, 2], [1000, 1500, 2000]]  # Example initial quantities for each AMM
initial_quantities = torch.tensor(quantities, requires_grad=False)

# Define Delta for the first n-1 AMMs only, since the last one will be derived
Delta_incomplete = torch.zeros((n-1, k), requires_grad=True)

def get_Delta(Delta_incomplete):
    assert Delta_incomplete.shape == (n-1, k), f"Delta_incomplete should have shape {(n-1, k)}"
    last_row_adjustment = -torch.sum(Delta_incomplete, axis=0, keepdims=True)
    # Combine adjustments to get the full Delta matrix
    return torch.cat((Delta_incomplete, last_row_adjustment), axis=0)

# Objective function considering the new Delta definition
def utilities(Delta):
    assert Delta.shape == (n, k), f"Delta should have shape {(n, k)}"
    # Calculate adjustments for the last AMM as the negation of the sum of the others
    new_quantities = initial_quantities + Delta
    products = new_quantities.prod(dim=1)  # Product of token quantities in each AMM
    return products

# get the gradient of utility w.r.t Delta
def get_gradient(Delta):
    assert Delta.shape == (n, k), f"Delta should have shape {(n, k)}"
    utilities_per_amm = utilities(Delta)
    # print(f"Delta[0]: {Delta[0]}")
    # print(f"utilities_per_amm[0]: {utilities_per_amm[0]}")
    gradients_per_amm = []
    for i in range(n):
        # if i < n - 1:
        #     grad_output = torch.autograd.grad(
        #         utilities_per_amm, 
        #         Delta, 
        #         grad_outputs=torch.ones_like(utilities_per_amm),
        #         retain_graph=True
        #     )
        #     print(grad_output)
        #     grad = grad_output[0]
        # else:
        #     # For the last AMM, compute gradient with respect to the negation of the sum of previous Deltas
        #     grad = torch.autograd.grad(utilities_per_amm[i], Delta[:-1], retain_graph=True)[0]
        #     grad = -torch.sum(grad, dim=0)  # Sum and negate to get the gradient for the last row
        # print(grad)
        # # HERE
        grad_outputs = torch.zeros_like(utilities_per_amm)
        grad_outputs[i] = 1  # Set 1 for the i-th utility, zeros elsewhere
        grad = torch.autograd.grad(utilities_per_amm, Delta, grad_outputs=grad_outputs,
                                retain_graph=True)[0][i]
        gradients_per_amm.append(grad)  # Assign gradient to corresponding Delta
        # gradients_per_amm.append(grad)
    # print(f"gradients_per_amm: {gradients_per_amm}")
    # print(f"stacked: {torch.stack(gradients_per_amm)}")
    return torch.stack(gradients_per_amm)
    # this has shape (n, k)

def cosine_utility(cosine_similarities):
    return cosine_similarities.sum()

def cosine_difference_loss(cosine_similarities):
    return cosine_similarities.var()

def cosine_similarities(grads, Delta) -> torch.Tensor:
    assert grads.shape == (n, k), f"grads should have shape {(n, k)}"
    assert Delta.shape == (n, k), f"Delta should have shape {(n, k)}"
    similarities = grads * Delta / torch.linalg.norm(grads, dim=1)
    return similarities.sum(dim=1)
    cosines = []
    for i in range(n):
        grad = grads[i]
        cos_times_v = torch.dot(grad, Delta[i]) / (torch.linalg.vector_norm(grad))
        cosines.append(cos_times_v)
    return cosines


def integrated_cosine_similarity(Delta):
    N = 20
    cosines = torch.zeros(Delta.shape[0])
    for i in range(N):
        curr_Delta = i/N * Delta
        grads = get_gradient(curr_Delta)
        cosines += cosine_similarities(grads, Delta) * (1/N)
    return cosines



# Define the optimizer
optimizer = optim.Adam([Delta_incomplete], lr=0.02)

# Training loop
for i in range(5000):
    optimizer.zero_grad()
    Delta = get_Delta(Delta_incomplete)
    # print(f"Delta: {Delta.shape}")
    # print(f"gradient: {get_gradient(Delta).shape}")
    # print(f"shapes: {Delta.shape}, {initial_quantities.shape}, {get_gradient(Delta).shape}")
    # cos_similarities = cosine_similarities(get_gradient(Delta), Delta)
    cos_similarities_integrated = integrated_cosine_similarity(Delta)
    cos_utility = 1000*cos_similarities_integrated.sum()
    cos_utility = utilities(Delta).sum() / 10
    cos_loss = 1000*cos_similarities_integrated.var()
    cos_loss = 0
    relu_loss = 100*torch.sum(torch.relu(-initial_quantities - Delta))
    loss = 0
    loss -= cos_utility
    loss += cos_loss
    loss += relu_loss
    quantities = initial_quantities + Delta
    prices = quantities[:, 0] / quantities[:, 1]
    # price_diff_loss = prices.var()
    # loss += 10000*price_diff_loss
    if i % 100 == 0:
        print(f"step {i} Cos utility: {cos_utility} | Cos loss: {cos_loss} | ReLU loss: {relu_loss} | prices: {prices}")
    loss.backward()
    optimizer.step()
    # if i % 100 == 0:
    #     print(f"Iteration {i}, loss: {loss.item()}")

    # if i % 100 == 0:
    #     print(f"Utility: {utility}")
    #     print(f"Delta: {Delta}")
    #     grads = get_gradient(Delta)
    #     print(f"Gradients: {grads}")
    #     cosines = cosine_similarities(grads, Delta)
    #     print(f"Cosine similarities: {cosines}")
    #     print("")

final_delta = get_Delta(Delta_incomplete)
final_grad = get_gradient(final_delta)
final_quantities = initial_quantities + final_delta
print(f"Final Delta: {final_delta}")
print(f"Final Utility: {utilities(final_delta)}")
print(f"Final Gradient: {final_grad}")
print(f"Final Cosine similarity integral: {integrated_cosine_similarity(final_delta)}")
print(f"Final Cosine utility: {cosine_utility(integrated_cosine_similarity(final_delta))}")
print(f"Final Cosine loss: {cosine_difference_loss(integrated_cosine_similarity(final_delta))}")
print(f"Final ReLU loss: {torch.sum(torch.relu(-initial_quantities - final_delta))}")

print(f"Final balances: {initial_quantities + final_delta}")
print(f"Final utilities: {utilities(final_delta)}")

print(f"final prices: {final_quantities[:, 0] / final_quantities[:, 1]}")


step 0 Cos utility: 260.0 | Cos loss: 0 | ReLU loss: 0.0 | prices: tensor([0.5000, 1.5000], grad_fn=<DivBackward0>)
step 100 Cos utility: 275.0952453613281 | Cos loss: 0 | ReLU loss: 0.0 | prices: tensor([0.4416, 1.4767], grad_fn=<DivBackward0>)
step 200 Cos utility: 292.7398681640625 | Cos loss: 0 | ReLU loss: 0.0 | prices: tensor([0.3602, 1.4567], grad_fn=<DivBackward0>)
step 300 Cos utility: 313.13653564453125 | Cos loss: 0 | ReLU loss: 0.0 | prices: tensor([0.2458, 1.4391], grad_fn=<DivBackward0>)
step 400 Cos utility: 336.4225769042969 | Cos loss: 0 | ReLU loss: 0.0 | prices: tensor([0.0796, 1.4232], grad_fn=<DivBackward0>)
step 500 Cos utility: 355.5625915527344 | Cos loss: 0 | ReLU loss: 0.0 | prices: tensor([0.0050, 1.3765], grad_fn=<DivBackward0>)
step 600 Cos utility: 371.23687744140625 | Cos loss: 0 | ReLU loss: 0.0 | prices: tensor([1.2688e-03, 1.3196e+00], grad_fn=<DivBackward0>)
step 700 Cos utility: 386.41680908203125 | Cos loss: 0 | ReLU loss: 0.0 | prices: tensor([0.00